
# Projet Maison 2

In [46]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [47]:
# chargement et traitement des données
GEO = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")

# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
# - cp_ville : Code Postal + " " + "Commune"

In [48]:
GEO[['lat', 'lon']] = GEO["geo_point_2d"].str.extract("(.*), (.*)").astype(float)

In [49]:
GEO.head(1)

Code Postal             Commune    Département         Région  \
Code INSEE                                                                 
31080            31350  BOULOGNE-SUR-GESSE  HAUTE-GARONNE  MIDI-PYRENEES   

                      Statut  Altitude Moyenne  Superficie  Population  \
Code INSEE                                                               
31080       Chef-lieu canton             301.0      2470.0         1.6   

                             geo_point_2d  \
Code INSEE                                  
31080       43.2904403081, 0.650641474176   

                                                    geo_shape       lat  \
Code INSEE                                                                
31080       {"type": "Polygon", "coordinates": [[[0.671852...  43.29044   

                 lon  
Code INSEE            
31080       0.650641

In [50]:
GEO['cp_ville'] = GEO['Code Postal'] + ' ' + GEO['Commune']

In [51]:
GEO.head(1)

Code Postal             Commune    Département         Région  \
Code INSEE                                                                 
31080            31350  BOULOGNE-SUR-GESSE  HAUTE-GARONNE  MIDI-PYRENEES   

                      Statut  Altitude Moyenne  Superficie  Population  \
Code INSEE                                                               
31080       Chef-lieu canton             301.0      2470.0         1.6   

                             geo_point_2d  \
Code INSEE                                  
31080       43.2904403081, 0.650641474176   

                                                    geo_shape       lat  \
Code INSEE                                                                
31080       {"type": "Polygon", "coordinates": [[[0.671852...  43.29044   

                 lon                  cp_ville  
Code INSEE                                      
31080       0.650641  31350 BOULOGNE-SUR-GESSE

**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [52]:
# fonction recherche de ville
def search_city(lat, long):
    return GEO.loc[((lat - GEO['lat'])**2 + (long - GEO['lon'])**2).idxmin()]['cp_ville']    

In [53]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn/60 + sec/3600    

In [54]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

'71330 BOSJEAN'

In [55]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [56]:
import json
GEO_SHAPE = GEO['geo_shape'].apply(json.loads).apply(pd.Series)
GEO_SHAPE

type                                        coordinates
Code INSEE                                                            
31080       Polygon  [[[0.671852008023359, 43.268343740258835], [0....
11143       Polygon  [[[2.906996888482618, 42.895679017325214], [2....
43028       Polygon  [[[4.091674013520042, 45.10387299771701], [4.0...
78506       Polygon  [[[1.82961019583912, 48.496571987531524], [1.8...
84081       Polygon  [[[4.927176726246697, 43.90528366187598], [4.8...
80462       Polygon  [[[1.849459801983121, 50.17500092547019], [1.8...
40200       Polygon  [[[-0.710168187746476, 44.34474055229595], [-0...
17394       Polygon  [[[-0.66124517570293, 46.112107175022146], [-0...
...             ...                                                ...
15024       Polygon  [[[2.302124109031132, 45.19200821045956], [2.2...
76008       Polygon  [[[1.205457180575811, 49.8846111948466], [1.16...
02137       Polygon  [[[3.258096611944434, 49.05600500728458], [3.2...
74066       Polygon  [[[6.026228645474093, 46.0398190939394], [6.01...
67315       Polygon  [[[8.106826841916497, 48.94076044446545], [8.0...
07122       Polygon  [[[4.356584796725173, 44.54080039174845], [4.3...
01026       Polygon  [[[4.926593676348706, 46.304132622674715], [4....
50224       Polygon  [[[-0.997539314886753, 48.950278285217586], [-...

[36742 rows x 2 columns]

In [57]:
# GEO_SHAPE
# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
#GEO_SHAPE = pd.Series(dtype=object)

In [58]:
# GEO_SHAPE['type'].value_counts()

In [59]:
import numpy as np

In [89]:
dataframe = GEO_SHAPE['coordinates'].apply(lambda x: np.array(x, dtype = object)).apply(lambda y: np.shape(y)[0])
dataframe.loc[GEO_SHAPE[GEO_SHAPE['type'] == 'Polygon'].index][dataframe == 2].count()

10

In [60]:
# value_counts des valeurs "type"
def get_types():
    return GEO_SHAPE['type'].value_counts()    

In [61]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
    dataframe = GEO_SHAPE['coordinates'].apply(lambda x: np.array(x, dtype = object)).apply(lambda y: np.shape(y)[0])
    return dataframe.value_counts()    

In [62]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    dataframe = GEO_SHAPE['coordinates'].apply(lambda x: np.array(x, dtype = object)).apply(lambda y: np.shape(y)[0])
    return GEO.loc[dataframe.idxmax()]['cp_ville']

In [90]:
# commune constituée du plus grand nombre de polygones
def get_nb_cities_2_polygons():
    dataframe = GEO_SHAPE['coordinates'].apply(lambda x: np.array(x, dtype = object)).apply(lambda y: np.shape(y)[0])
    return dataframe.loc[GEO_SHAPE[GEO_SHAPE['type'] == 'Polygon'].index][dataframe == 2].count()

In [91]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [92]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 2.013s

OK
